In [1]:
from src.mtal.data_collect import get_spot_pairs

cryptos = get_spot_pairs()


//@version=5
indicator("Manual Trades", overlay=true)

var int[] entryDates = array.new_int()
var int[] exitDates = array.new_int()

array.push(entryDates, 1513555199999)
array.push(entryDates, 1517788799999)
array.push(exitDates, 1519603199999)
array.push(exitDates, 1520812799999)

for i = 0 to array.size(exitDates) - 1
    label.new(x=array.get(entryDates, i), xloc=xloc.bar_time, y=close, yloc=yloc.belowbar, color=color.green, textcolor=color.white, style=label.style_label_up)
    label.new(x=array.get(exitDates, i), xloc=xloc.bar_time, y=close, yloc=yloc.abovebar, color=color.red, textcolor=color.white, style=label.style_label_down)
    
           


In [2]:
from src.mtal.data_collect import get_pair_df

# df = get_pair_df(pair="BTCUSDT")
df = get_pair_df(pair="BTCUSDT", limit=1000, frequency="1w")
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time
0,2017-08-14,4261.48,4485.39,3850.00,4086.29,2843.431426,2017-08-20 23:59:59.999
1,2017-08-21,4069.13,4453.91,3400.00,4310.01,4599.396629,2017-08-27 23:59:59.999
2,2017-08-28,4310.01,4939.19,4124.54,4509.08,4753.843376,2017-09-03 23:59:59.999
3,2017-09-04,4505.00,4788.59,3603.00,4130.37,6382.787745,2017-09-10 23:59:59.999
4,2017-09-11,4153.62,4394.59,2817.00,3699.99,8106.705127,2017-09-17 23:59:59.999


In [9]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ema_cross_backtest import EMACrossBacktester 

short_ema =2 
long_ema = 4
tester = EMACrossBacktester(
    short_ema=short_ema, long_ema=long_ema, data=df
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
print(generate_pinescript(entry_dates, exit_dates))


BacktestResults(pnl=47919.52004889779, pnl_percentage=46.919520048897795, max_drawdown=-0.17942207786354494, win_rate=0.41379310344827586, average_return=0.26760179382596816, trade_number=29, entry_dates=[Timestamp('2017-08-28 00:00:00'), Timestamp('2017-10-02 00:00:00'), Timestamp('2018-03-05 00:00:00'), Timestamp('2018-04-16 00:00:00'), Timestamp('2018-07-23 00:00:00'), Timestamp('2018-09-03 00:00:00'), Timestamp('2019-01-07 00:00:00'), Timestamp('2019-02-11 00:00:00'), Timestamp('2019-08-05 00:00:00'), Timestamp('2019-09-16 00:00:00'), Timestamp('2019-10-28 00:00:00'), Timestamp('2020-01-13 00:00:00'), Timestamp('2020-04-13 00:00:00'), Timestamp('2020-07-13 00:00:00'), Timestamp('2020-10-12 00:00:00'), Timestamp('2021-05-03 00:00:00'), Timestamp('2021-07-26 00:00:00'), Timestamp('2021-10-04 00:00:00'), Timestamp('2022-02-14 00:00:00'), Timestamp('2022-03-21 00:00:00'), Timestamp('2022-07-25 00:00:00'), Timestamp('2022-09-12 00:00:00'), Timestamp('2022-10-24 00:00:00'), Timestamp('20

In [10]:
from itertools import combinations
from src.mtal.backtesting.ema_cross_backtest import EMACrossBacktester

# Charger les données

# Plage des valeurs pour EMAs
ema_values = range(1, 60)  # Exemple de 1 à 49

# Générer des combinaisons où short_ema < long_ema
combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

# Dictionnaire pour stocker les résultats
results = {}

# Tester toutes les combinaisons
for short_ema, long_ema in combinations:
    tester = EMACrossBacktester(short_ema=short_ema, long_ema=long_ema, data=df)
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

# Trouver la meilleure combinaison basée sur un critère spécifique
best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


In [ ]:
entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in best_result.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in best_result.exit_dates]

print(generate_pinescript(entry_dates, exit_dates))


        //@version=5
        indicator("Manual Trades", overlay=true)

        var int[] entryDates = array.new_int()
        var int[] exitDates = array.new_int()

     
        array.push(entryDates, 1503878399999)
        array.push(entryDates, 1506902399999)
        array.push(entryDates, 1520207999999)
        array.push(entryDates, 1523836799999)
        array.push(entryDates, 1532303999999)
        array.push(entryDates, 1535932799999)
        array.push(entryDates, 1546819199999)
        array.push(entryDates, 1549843199999)
        array.push(entryDates, 1564963199999)
        array.push(entryDates, 1568591999999)
        array.push(entryDates, 1572220799999)
        array.push(entryDates, 1578873599999)
        array.push(entryDates, 1586735999999)
        array.push(entryDates, 1594598399999)
        array.push(entryDates, 1602460799999)
        array.push(entryDates, 1619999999999)
        array.push(entryDates, 1627257599999)
        array.push(entryDates, 1633305599999)
